**改編自〈sovits3.0一键脚本（小狼躺平了，所以是深夜诗人修改版本,已更新32k/48k分支切换）.ipynb〉**

https://colab.research.google.com/drive/1rCUOOVG7-XQlVZuWRAj5IpGrMM8t07pE?usp=sharing

做了大幅(?)調整使其可用在vast.ai的機台上頭（運作映像為nvidia/cuda:12.0.1-runtime-ubuntu22.04 `00602486357e8c88c07157ddcd8fb2d7`），包含各種版本差異造成的坑。

原Repo `innnky/so-vits-svc` 因為某些因素不見了，所以使用我自己的fork，預訓練模型採用 `TachibanaKimika/so-vits-svc-4.0-models` 提供的模型。

註解繁中化。

In [1]:
# 查看顯示卡
!nvidia-smi

Tue Mar  7 09:20:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:03:00.0 Off |                  N/A |
|  0%   42C    P8     4W / 420W |      1MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from pathlib import Path

root = Path('/root')
sovits = root / 'so-vits-svc'

In [5]:
#@title Clone github Repo
!git clone https://github.com/gimi65536/so-vits-svc -b 4.0
%cd {sovits}

fatal: destination path 'so-vits-svc' already exists and is not an empty directory.
/root/so-vits-svc


In [ ]:
# 安裝Dependencies
!apt-get install build-essential zip -y
!pip install pyworld praat-parselmouth librosa==0.9.2 fairseq


In [ ]:
# 下載必要模型文件
!wget -P hubert/ https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/others/checkpoint_best_legacy_500.pt

# 資料集預處理預處理

該處理腳本可以一次性預處理多位角色，並且生成多角色的filelists以及對應的配置文件

只需要將你的資料集按照以下文件結構放到dataset_raw資料夾下


```
dataset_raw
├───speaker0
│   ├───xxx1-xxx1.wav
│   ├───...
│   └───Lxx-0xx8.wav
└───speaker1
    ├───xx2-0xxx2.wav
    ├───...
    └───xxx7-xxx007.wav

```




In [3]:
# 設定訓練資料集備份與還原的路徑，預設為/root
DATA_PATH = root

In [ ]:
# 資料集名稱（壓縮檔名稱不帶zip）
DATASETNAME = "mika"  #@param {type:"string"}
# 壓縮檔路徑
ZIP_PATH = root
ZIP_NAME = ZIP_PATH / DATASETNAME

!unzip -d {sovits / 'dataset_raw'} {ZIP_NAME}.zip

In [10]:
# 重取樣到44100Hz
!python3 resample.py

./dataset_raw/mika_voice
69it [00:00, 264.65it/s]


In [11]:
# 劃分訓練集、生成配置文件
!python3 preprocess_flist_config.py

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 159.01it/s]
Writing ./filelists/train.txt
100%|███████████████████████████████████████| 65/65 [00:00<00:00, 711827.05it/s]
Writing ./filelists/val.txt
100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 50840.05it/s]
Writing ./filelists/test.txt
100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 52758.54it/s]
Writing configs/config.json


In [14]:
# 生成hubert和f0
!python3 preprocess_hubert_f0.py

[69]
Loading hubert for content...
load model(s) from hubert/checkpoint_best_legacy_500.pt
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/root/so-vits-svc, universal_newlines=False, shell=None, istream=None)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/root/so-vits-svc, universal_newlines=False, shell=None, istream=None)
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
INFO:fairseq.tasks.hubert_pretraining:current directory is /root/so-vits-svc
INFO:fairseq.tasks.hubert_pretraining:HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
INFO:fairseq.models.hubert.hubert:HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_

In [ ]:
# 至此，數據集預處理製作完畢，將資料集和相關文件保存到so-vits-svc資料夾中並自行下載，方便下次訓練使用

# 壓縮dataset資料夾
!zip -r dataset.zip dataset
# 自訂壓縮檔名稱（不含zip）
dataset_name_drive = "44kmika_dataset"
DATASET_PATH_DRIVE = DATA_PATH / dataset_name_drive
!mkdir -p {DATASET_PATH_DRIVE}

!cp dataset.zip "{DATASET_PATH_DRIVE}"
!cp configs/config.json "{DATASET_PATH_DRIVE}"
!cp filelists/train.txt "{DATASET_PATH_DRIVE}"
!cp filelists/test.txt "{DATASET_PATH_DRIVE}"
!cp filelists/val.txt "{DATASET_PATH_DRIVE}"

# 自行下載壓縮檔{DATA_PATH / dataset_name_drive}.zip

In [4]:
# 已經預處理過資料集的話，就可以跳過預處理部分，直接解壓縮處理過的資料以及配置文件

# 壓縮檔名稱（不含zip）
back_up_name = "44kmika_dataset"
BACK_UP_DATASET_PATH = DATA_PATH / back_up_name
!unzip {BACK_UP_DATASET_PATH / 'dataset.zip'} -d .
!cp {BACK_UP_DATASET_PATH / 'config.json'} configs/config.json 
!cp {BACK_UP_DATASET_PATH / 'val.txt'} filelists/val.txt
!cp {BACK_UP_DATASET_PATH / 'train.txt'} filelists/train.txt
!cp {BACK_UP_DATASET_PATH / 'test.txt'} filelists/test.txt

# 將之前訓練過的模型G_xx.pth與D_xx.pth放到 logs/44k 底下……

Archive:  /root/48kmika_dataset/dataset.zip
replace /content/so-vits-svc/dataset/48k/mika_voice/183845.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


# 訓練

In [ ]:
# 如果是首次訓練，則需事先下載預訓練模型
!wget -P logs/44k/ https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/others/G_0.pth
!wget -P logs/44k/ https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/others/D_0.pth

In [ ]:
!python3 train.py -c configs/config.json -m 44k